In [28]:
import numpy as np 
import pandas as pd
from datetime import date, timedelta
import glob
import xarray as xr
import rioxarray 
import subprocess
from pathlib import Path

In [ ]:
! rm -rf $outFpath ||:

In [ ]:

data_folder = Path("source_data/text_files/")

file_to_open = data_folder / "raw_data.txt"

f = open(file_to_open)


In [29]:
# define directory for tif files
clim_dir = Path('/nfs/attic/dfh/data/snodas/snodas_tif/clim/')
hs_dir = Path('/nfs/attic/dfh/data/snodas/snodas_tif/daily_Hs/')

In [3]:
sdate = date(2003,1,1)   # start date
edate = date(2020,12,31)   # end date
dates = pd.date_range(sdate,edate,freq='d')
# Create DataFrame
df = pd.DataFrame({'day':dates.day,'month':dates.month})
data = df.drop_duplicates()
data.reset_index(inplace=True,drop=True)

In [4]:
hs_dir = '/nfs/attic/dfh/data/snodas/snodas_tif/daily_Hs/'
for idx in data.index.tolist():
    print(f"{data.month[idx]:02d}"+f"{data.day[idx]:02d}")
    geotiff_list = glob.glob(hs_dir+'us_ssmv11036tS__T0001TTNATS*'+f"{data.month[idx]:02d}"+f"{data.day[idx]:02d}"+'05HP001.tif',recursive = True)

    # Create variable used for time axis
    time_var = xr.Variable('count', np.arange(0,len(geotiff_list),1))

    # Load in and concatenate all individual GeoTIFFs
    geotiffs_da = xr.concat([xr.open_rasterio(i) for i in geotiff_list],
                            dim=time_var)

    # Covert our xarray.DataArray into a xarray.Dataset
    geotiffs_ds = geotiffs_da.to_dataset('band')

    # Rename the variable to a more useful name
    geotiffs_ds = geotiffs_ds.rename({1: 'Hs'})
    
    print('std')
    # calculate standard deviation
    std = geotiffs_ds.std(dim = 'count',keep_attrs=True)
    #specify crs
    std.rio.write_crs("epsg:4326", inplace=True)
    #define filename
    outFname = clim_dir+ f"{data.month[idx]:02d}"+f"{data.day[idx]:02d}"+'1036std.tif'
    std["Hs"].rio.to_raster(outFname)

    print('q1')
    # calculate 1st quantile
    q1 = geotiffs_ds.quantile(.25,dim = 'count',keep_attrs=True)
    #specify crs
    q1.rio.write_crs("epsg:4326", inplace=True)
    #define filename
    outFname = clim_dir+ f"{data.month[idx]:02d}"+f"{data.day[idx]:02d}"+'1036q1.tif'
    q1["Hs"].rio.to_raster(outFname)

    print('q3')
    # calculate 3rd quantile
    q3 = geotiffs_ds.quantile(.75,dim = 'count',keep_attrs=True)
    #specify crs
    q3.rio.write_crs("epsg:4326", inplace=True)
    #define filename
    outFname = clim_dir+ f"{data.month[idx]:02d}"+f"{data.day[idx]:02d}"+'1036q3.tif'
    q3["Hs"].rio.to_raster(outFname)

0101


KeyboardInterrupt: 

# create daily tif files

In [40]:
sdate = date(2020,6,30)   # start date
edate = date(2021,9,30)   # end date
dates = pd.date_range(sdate,edate,freq='d')
dates

DatetimeIndex(['2020-06-30', '2020-07-01', '2020-07-02', '2020-07-03',
               '2020-07-04', '2020-07-05', '2020-07-06', '2020-07-07',
               '2020-07-08', '2020-07-09',
               ...
               '2021-09-21', '2021-09-22', '2021-09-23', '2021-09-24',
               '2021-09-25', '2021-09-26', '2021-09-27', '2021-09-28',
               '2021-09-29', '2021-09-30'],
              dtype='datetime64[ns]', length=458, freq='D')

In [ ]:
i = 0
indate = dates[0]
# define path names
Fname = '/nfs/attic/dfh/data/snodas/snodas_download/'+indate.strftime("%Y")+'/'+indate.strftime("%m")+'_'+indate.strftime("%b")+'/Hs/us_ssmv11036tS__T0001TTNATS'+indate.strftime("%Y")+indate.strftime("%m")+indate.strftime("%d")+'05HP001.dat'
#hdr_base_path = '/nfs/attic/dfh/data/snodas/snodas_download/snodas_base_header.hdr'
HDRname = '/nfs/attic/dfh/data/snodas/snodas_download/'+indate.strftime("%Y")+'/'+indate.strftime("%m")+'_'+indate.strftime("%b")+'/Hs/us_ssmv11036tS__T0001TTNATS'+indate.strftime("%Y")+indate.strftime("%m")+indate.strftime("%d")+'05HP001.hdr'

#create temporary hdr files 

In [35]:
hs_dir = Path('/nfs/attic/dfh/Aragon2/Notebooks/assimilation_python/test/')

In [26]:
i = 0
indate = dates[0]
# define path names
Fname = '/nfs/attic/dfh/data/snodas/snodas_download/'+indate.strftime("%Y")+'/'+indate.strftime("%m")+'_'+indate.strftime("%b")+'/Hs/us_ssmv11036tS__T0001TTNATS'+indate.strftime("%Y")+indate.strftime("%m")+indate.strftime("%d")+'05HP001.dat'
#hdr_base_path = '/nfs/attic/dfh/data/snodas/snodas_download/snodas_base_header.hdr'
HDRname = '/nfs/attic/dfh/data/snodas/snodas_download/'+indate.strftime("%Y")+'/'+indate.strftime("%m")+'_'+indate.strftime("%b")+'/Hs/us_ssmv11036tS__T0001TTNATS'+indate.strftime("%Y")+indate.strftime("%m")+indate.strftime("%d")+'05HP001.hdr'

#create temporary hdr files 
#subprocess.call(f'cp {hdr_base_path} {HDRname}',shell=True)

# create .tif
output_tif = hs_dir+'us_ssmv11036tS__T0001TTNATS'+indate.strftime("%Y")+indate.strftime("%m")+indate.strftime("%d")+'05HP001.tif'
cmd = f'gdal_translate -of GTiff -a_srs "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs" -a_nodata -9999 -a_ullr -124.73333333333333 52.87500000000000 -66.94166666666667 24.95000000000000 {Fname} {output_tif}'
subprocess.call(cmd, shell=True)

#remove temporary files 
#subprocess.call(f'rm {HDRname}',shell=True)

1

In [42]:
# Load in and concatenate all individual GeoTIFFs
geotiffs_da = xr.open_rasterio(fileP)

# Covert our xarray.DataArray into a xarray.Dataset
geotiffs_ds = geotiffs_da.to_dataset('band')

# Rename the variable to a more useful name
geotiffs_ds = geotiffs_ds.rename({1: 'Hs'})

geotiffs_ds

RasterioIOError: /nfs/attic/dfh/data/snodas/snodas_download/2020/07_Jul/Hs/us_ssmv11036tS__T0001TTNATS2020070105HP001.dat: Permission denied

In [41]:
snodasP = Path('/nfs/attic/dfh/data/snodas/snodas_download/'+indate.strftime("%Y")+'/'+indate.strftime("%m")+'_'+indate.strftime("%b")+'/Hs/')
Fname = 'us_ssmv11036tS__T0001TTNATS'+indate.strftime("%Y")+indate.strftime("%m")+indate.strftime("%d")+'05HP001.dat'
fileP = snodasP / Fname
HDRname = 'us_ssmv11036tS__T0001TTNATS'+indate.strftime("%Y")+indate.strftime("%m")+indate.strftime("%d")+'05HP001.hdr'
hdrP = snodasP / HDRname
TIFname = 'us_ssmv11036tS__T0001TTNATS'+indate.strftime("%Y")+indate.strftime("%m")+indate.strftime("%d")+'05HP001.tif'
output_tif = hs_dir / TIFname
#!gdal_translate -of GTiff -a_srs "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs" -a_nodata -9999 -a_ullr -124.73333333333333 52.87500000000000 -66.94166666666667 24.95000000000000 $fileP $output_tif
#subprocess.call(cmd, shell=True)


In [32]:
snodasP = Path('/nfs/attic/dfh/data/snodas/snodas_download/'+indate.strftime("%Y")+'/'+indate.strftime("%m")+'_'+indate.strftime("%b")+'/Hs/')
snodasP

PosixPath('/nfs/attic/dfh/data/snodas/snodas_download/2020/07_Jul/Hs')

In [17]:
output_tif = hs_dir+'us_ssmv11036tS__T0001TTNATS'+indate.strftime("%Y")+indate.strftime("%m")+indate.strftime("%d")+'05HP001.tif'
output_tif

'test/us_ssmv11036tS__T0001TTNATS2020070105HP001.tif'

In [19]:
HDRname

'/nfs/attic/dfh/data/snodas/snodas_download/2020/07_Jul/Hs/us_ssmv11036tS__T0001TTNATS2020070105HP001.hdr'

In [23]:
!pwd

/nfs/attic/dfh/Aragon2/Notebooks/assimilation_python
